In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import re

# Selenium 드라이버 설정
driver = webdriver.Chrome()

# CSV 파일 경로
csv_file_path = 'G:\\내 드라이브\\project_poster\\posters_data.csv'

# 데이터프레임 초기화
data = {
    'img': [],   # 이미지 파일명
    'title': [], # 영화 제목
    'genre': []  # 영화 장르
}

# 1. URL 방문
page_url = 'https://www.kmdb.or.kr/db/have/detailSearch/imageSearch'
driver.get(page_url)
time.sleep(1)

# 2. 첫 번째 버튼 클릭 (필요에 따라)
try:
    first_button = driver.find_element(By.XPATH, "/html/body/div[2]/div[5]/div[2]/section/div/div[3]/div[3]/span/a")
    first_button.click()
    time.sleep(1)  # 클릭 후 페이지 로딩 대기
except Exception as e:
    print(f"첫 번째 버튼 클릭 중 오류 발생: {e}")

# 이미지를 순서대로 저장할 파일 넘버
image_number = 0
sleep_time = 1

# 이미 선택한 이미지들을 저장할 세트 (중복 방지용)
selected_images = set()

# 5번 반복 (필요 시 더 많은 페이지를 반복할 수 있음)
for _ in range(5):
# for _ in range(len(selected_images)):
    # 3. 이미지 목록 가져오기 (매번 반복문 안에서 다시 가져오기)
    posters = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//span[contains(@style, 'background-image')]"))
    )

    for span in posters:
        try:
            # 요소를 새로 가져옴 (stale element 방지)
            style = span.get_attribute('style')
            img_url_match = re.search(r'url\((.*?)\)', style)

            if img_url_match:
                img_url = img_url_match.group(1).strip("'\"")

                # 중복 방지: 이미 선택된 이미지 URL은 스킵
                if img_url in selected_images:
                    continue

                # URL이 처음 선택된 경우만 처리
                if img_url.startswith('http://file.koreafilm.or.kr/poster/'):
                    print(f"Found image URL: {img_url}")
                    selected_images.add(img_url)  # 중복 방지를 위해 선택된 이미지를 기록
                    
                    try:
                        # 이미지를 클릭
                        span.click()
                        time.sleep(2)  # 클릭 후 페이지 로딩 대기

                        # 4. 영화 제목 클릭 후 수집
                        title_button = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div[5]/div[2]/form/section/div[1]/div[2]/div[1]/a/div"))
                        )
                        title = title_button.text
                        data['title'].append(title)
                        print(f"Title collected: {title}")
                        
                        # 4.1 추가: 영화 제목 클릭 후 페이지 로딩 대기
                        x_button = driver.find_element(By.XPATH, "/html/body/div[4]/div[2]/div[1]/a")
                        x_button.click()
                        time.sleep(1)  # 클릭 후 페이지 로딩 대기

                        # 5. 장르 페이지로 이동 (클릭)
                        genre_button = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div[5]/div[2]/form/section/div[2]/div[1]/div[1]/div/div[2]/a"))
                        )
                        genre_button.click()
                        time.sleep(1)  # 클릭 후 페이지 로딩 대기

                        # 6. 장르 수집
                        genre_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div[5]/div[2]/form/section/div[5]/div[4]/dl/dd[1]"))
                        )
                        genre_text = genre_element.text

                        # '#' 제거하고, 콤마로 구분하여 저장
                        genres = [g.strip() for g in genre_text.split('#') if g]
                        data['genre'].append(', '.join(genres))
                        print(f"Genre collected: {genres}")

                        # 이미지 파일명 저장
                        image_filename = f'poster_{image_number}.jpg'
                        data['img'].append(image_filename)
                        image_number += 1
                        
                    except Exception as e:
                        print(f"이미지 클릭 및 정보 수집 중 오류 발생: {e}")
                        driver.back()  # 오류가 발생하면 페이지를 되돌리기
        except Exception as e:
            print(f"포스터 처리 중 오류 발생: {e}")

    # CSV 파일로 저장
    df = pd.DataFrame(data)
    df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')
    print("CSV 파일이 저장되었습니다:", csv_file_path)
    
    # 5. 장르 페이지로 이동 후 다시 리스트로 돌아가기
    max_back_attempts = 10  # 최대 뒤로 가기 시도 횟수
    back_attempts = 0

    while back_attempts < max_back_attempts:
        try:
            # 뒤로 가기 버튼 클릭
            back_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div[5]/div[2]/div[1]/div/div/a"))  # 뒤로 가기 버튼의 XPATH
            )
            back_button.click()
            time.sleep(2)  # 버튼 클릭 후 페이지 로딩 대기

            # 특정 URL의 일부가 포함되었는지 확인
            WebDriverWait(driver, 10).until(
                EC.url_contains('imageSearch')  # 돌아와야 할 URL의 일부
            )

            # URL이 확인되면 반복 종료
            print("페이지가 성공적으로 돌아왔습니다.")
            break  # 페이지가 돌아오면 while 루프 종료

        except Exception as e:
            print(f"뒤로 가기 버튼 클릭 중 오류 발생: {e}")
            back_attempts += 1
            if back_attempts >= max_back_attempts:
                print("최대 뒤로 가기 시도 횟수 초과. 반복을 중단합니다.")

# 드라이버 종료
driver.quit()

Found image URL: http://file.koreafilm.or.kr/poster/00/00/81/DPK001889_01.jpg
Title collected: 무궁화꽃이 피었습니다 
Genre collected: ['드라마', '사회물(경향)', '미스터리', '전쟁']
포스터 처리 중 오류 발생: Message: stale element reference: stale element not found
  (Session info: chrome=128.0.6613.138); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7F4499412+29090]
	(No symbol) [0x00007FF7F440E239]
	(No symbol) [0x00007FF7F42CB1DA]
	(No symbol) [0x00007FF7F42DDBE9]
	(No symbol) [0x00007FF7F42D2954]
	(No symbol) [0x00007FF7F42D2AD9]
	(No symbol) [0x00007FF7F42D0787]
	(No symbol) [0x00007FF7F42D43A8]
	(No symbol) [0x00007FF7F436772C]
	(No symbol) [0x00007FF7F43466EA]
	(No symbol) [0x00007FF7F43665A2]
	(No symbol) [0x00007FF7F4346493]
	(No symbol) [0x00007FF7F43109D1]
	(No symbol) [0x00007FF7F4311B31]
	GetHandleVerifier [0x00007FF7F47B871D+3302573]
	GetHandleVerifier [0x0000